<a href="https://colab.research.google.com/github/nisarahamedk/Kaggle_Notebooks/blob/master/melanoma_classification/efficient_net_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EfficientNet on TPU

---



### Install packages

In [0]:
%%capture
!pip install -U efficientnet

### Imports

In [0]:
from pathlib import Path

import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import wandb
from wandb.keras import WandbCallback

import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets

### WandB Login

In [0]:
!wandb login f137298421da563b24639d1287dd3ce5da537814

In [0]:
wandb.init(project="kaggle-melanoma")

## TPU Config

Detect hardware and return appropriate distribution strategy

In [0]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # no parameter needed for TPU_NAME env variable is set. This is the case for Kaggle
    print("Running on TPU: ", tpu.master())
except ValueError:
    tpu = None

In [0]:
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default strategy with the available hw
    strategy = tf.distribute.get_strategy()
    
print("REPLICAS: ", strategy.num_replicas_in_sync)

## Dataset

In [0]:
train_files = tf.io.gfile.glob(GCS_DS_PATH + "/tfrecords/train*")
test_files = tf.io.gfile.glob(GCS_DS_PATH + "/tfrecords/test*")

## tf.Dataset pipeline

Building the complete tfrecord -> model data pipeline

In [0]:
train_dataset = tf.data.TFRecordDataset(train_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.TFRecordDataset(test_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)

Checking the feature discription of the tfreceord files

In [0]:
# train set
for item in train_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(item.numpy())
    print(example)

We have features: "image", "image_name" and "target"

In [0]:
# test set
for item in test_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(item.numpy())
    print(example)

We have "image" and "image_name" for the test dataset

#### Feature description

In [0]:
train_feature_desc = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "target": tf.io.FixedLenFeature([], tf.int64),
}

test_feature_desc = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "image_name": tf.io.FixedLenFeature([], tf.string),
}

Using the above feature description,

Lets load the dataset from the tfrecord files  
1. Process it using the feature description.
2. Decode each sample into an image.
3. return image, target pairs
4. Read from bottom to top

In [0]:
def decode_image(image_data):
    image_size = [1024, 1024]
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*image_size, 3])  # explicit size needed for TPU
    image = tf.image.resize(image, [512, 512])
    return image
                           
def parse_test_example(example):
    """ function to parse each example read from the test tfrecord file"""
    example = tf.io.parse_single_example(example, test_feature_desc)
    image = decode_image(example["image"])
    image_name = example["image_name"]
    return image, image_name

def parse_train_example(example):
    """ function to parse each example read from the train tfrecord file"""
    example = tf.io.parse_single_example(example, train_feature_desc)
    image = decode_image(example["image"])
    label = tf.cast(example["target"], tf.int32)
    return image, label

def load_dataset_from_tfrecord(filenames, is_training=True, ordered=False):
    
    # Since we are reading dataset from multiple files. and we dont care about the order.
    # set deterministic reading to False.
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
    dataset.with_options(ignore_order)
    
    # parse each example with feature description
    dataset = dataset.map(parse_train_example if is_training else parse_test_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    return dataset
    

Data augmentation for the image

In [0]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

Finally, the datapipeline function that puts these all together

In [0]:
batch_size = 16 * strategy.num_replicas_in_sync
def get_training_dataset():
    dataset = load_dataset_from_tfrecord(train_files, is_training=True)
    dataset = dataset.map(data_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [0]:
def get_test_dataset(ordered=False):
    dataset = load_dataset_from_tfrecord(test_files, is_training=False, ordered=ordered)
    dataset = dataset.batch(batch_size)
    dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

Sanity check

In [0]:
print("*** Training set shapes *****")
for image, label in get_training_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
    
print("*** Training set labels: ", label.numpy())

print("*** Test set shape ***")
for image, image_name in get_test_dataset().take(3):
    print(image.numpy().shape, image_name.numpy().shape)
print("*** Test set image_name: ", image_name.numpy().astype("U"))

### Model - Efficient Net

In [0]:
with strategy.scope():
    model = keras.Sequential([
        efn.EfficientNetB3(
            input_shape=[512, 512, 3],
            weights="imagenet",
            include_top=False,
        ),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(1, activation="sigmoid")
    ])

In [0]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

### Training

In [0]:
history = model.fit(get_training_dataset(), epochs=5, callbacks=[WandbCallback()])

### Submission

In [0]:
test_ds = get_test_dataset(ordered=True)

print("Computing predictions...")
test_image_ds = test_ds.map(lambda image, image_name: image)
probs = model.predict(test_image_ds).flatten()
print(probs)

In [0]:
import re
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [0]:
print("Generating submission file")
num_test_images = count_data_items(test_files)
test_image_names_ds = test_ds.map(lambda image, image_name: image_name).unbatch()

test_image_names = next(iter(test_image_names_ds.batch(num_test_images))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_image_names, probs]), fmt=['%s', '%f'], delimiter=',', header='image_name,target', comments='')
!head submission.csv